In [1]:
from __future__ import absolute_import, division, print_function

!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print (device_name)

    100% |████████████████████████████████| 332.1MB 81kB/s 
    100% |████████████████████████████████| 3.0MB 6.5MB/s 
    100% |████████████████████████████████| 419kB 11.8MB/s 
    100% |████████████████████████████████| 61kB 21.8MB/s 
/device:GPU:0


In [0]:
import matplotlib.pyplot as plt
% matplotlib inline
import numpy as np
import pandas as pd
import time
import random
from collections import Counter
import math

In [3]:
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
batch_size = 64
emb_size = 200
hidden_size = 512
lr = 1e-3

In [0]:
train_en_path = '/content/drive/My Drive/Colab Notebooks/cs11747/data/para/train.en.txt' # train set
train_ja_path = '/content/drive/My Drive/Colab Notebooks/cs11747/data/para/train.ja.txt' # train set
dev_en_path = '/content/drive/My Drive/Colab Notebooks/cs11747/data/para/dev.en.txt' # dev set
dev_ja_path = '/content/drive/My Drive/Colab Notebooks/cs11747/data/para/dev.ja.txt' # dev set
test_en_path = '/content/drive/My Drive/Colab Notebooks/cs11747/data/para/test.en.txt' 
test_ja_path = '/content/drive/My Drive/Colab Notebooks/cs11747/data/para/test.ja.txt' 

In [0]:
def read_data(file):
    with open(file, "r") as f:
        data = []
        for line in f:
            words = line.lower().strip()
            words = '<start> ' + words + ' <end>'
            data.append(words)
        return data

In [0]:
train_en = read_data(train_en_path)
train_ja = read_data(train_ja_path)
dev_en = read_data(dev_en_path)
dev_ja = read_data(dev_ja_path)
test_en = read_data(test_en_path)
test_ja = read_data(test_ja_path)

In [0]:
tokenizer_en = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;=?@[\]^_`{|}~ ', oov_token='<unk>') 
tokenizer_en.fit_on_texts(train_en)
train_seq_en = tokenizer_en.texts_to_sequences(train_en)
dev_seq_en = tokenizer_en.texts_to_sequences(dev_en)
test_seq_en = tokenizer_en.texts_to_sequences(test_en)

In [9]:
w2i_en = tokenizer_en.word_index
i2w_en = dict([(value, key) for (key, value) in w2i_en.items()])
nwords_en = len(w2i_en)
print (nwords_en)

7007


In [0]:
tokenizer_ja = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;=?@[\]^_`{|}~ ', oov_token='<unk>') 
tokenizer_ja.fit_on_texts(train_ja)
train_seq_ja = tokenizer_ja.texts_to_sequences(train_ja)
dev_seq_ja = tokenizer_ja.texts_to_sequences(dev_ja)
test_seq_ja = tokenizer_ja.texts_to_sequences(test_ja)

In [11]:
w2i_ja = tokenizer_ja.word_index
i2w_ja = dict([(value, key) for (key, value) in w2i_ja.items()])
nwords_ja = len(w2i_ja)
print (nwords_ja)

8055


In [12]:
w2i_ja['<unk>']

1

In [0]:
def train_gen(batch_size=batch_size):
    i = 0
    while i<len(train_seq_en):
        if i+batch_size < len(train_seq_en):
            batch_en = tf.keras.preprocessing.sequence.pad_sequences(train_seq_en[i:i+batch_size], padding='post')
            batch_ja = tf.keras.preprocessing.sequence.pad_sequences(train_seq_ja[i:i+batch_size], padding='post')
        else:
            batch_en = tf.keras.preprocessing.sequence.pad_sequences(train_seq_en[i:], padding='post')
            batch_ja = tf.keras.preprocessing.sequence.pad_sequences(train_seq_ja[i:], padding='post')
        i += batch_size
        yield batch_en, batch_ja

In [0]:
def dev_gen(batch_size=batch_size):
    i = 0
    while i<len(dev_seq_en):
        if i+batch_size < len(dev_seq_en):
            batch_en = tf.keras.preprocessing.sequence.pad_sequences(dev_seq_en[i:i+batch_size], padding='post')
            batch_ja = tf.keras.preprocessing.sequence.pad_sequences(dev_seq_ja[i:i+batch_size], padding='post')
        else:
            batch_en = tf.keras.preprocessing.sequence.pad_sequences(dev_seq_en[i:], padding='post')
            batch_ja = tf.keras.preprocessing.sequence.pad_sequences(dev_seq_ja[i:], padding='post')
        i += batch_size
        yield batch_en, batch_ja

In [0]:
def test_gen(batch_size=batch_size):
    i = 0
    while i<len(test_seq_en):
        if i+batch_size < len(test_seq_en):
            batch_en = tf.keras.preprocessing.sequence.pad_sequences(test_seq_en[i:i+batch_size], padding='post')
        else:
            batch_en = tf.keras.preprocessing.sequence.pad_sequences(test_seq_en[i:], padding='post')
        i += batch_size
        yield batch_en

In [0]:
class Encoder(tf.keras.Model):
    def __init__(self, nwords, embed_size, hidden_size):
        super(Encoder, self).__init__()
        self.embedding = tf.keras.layers.Embedding(nwords+1, embed_size, trainable=True)
        self.gru = tf.keras.layers.GRU(hidden_size, 
                                       return_sequences=True, 
                                       return_state=True, 
                                       recurrent_initializer='glorot_uniform')

    def call(self, x):
        x = self.embedding(x)
        # passing all time steps to the GRU
        outputs, state = self.gru(x)

        return outputs, state

In [0]:
encoder = Encoder(nwords_en, emb_size, hidden_size)

In [0]:
class Decoder(tf.keras.Model):
    def __init__(self, nwords, embed_size, hidden_size):
        super(Decoder, self).__init__()
        self.embedding = tf.keras.layers.Embedding(nwords+1, embed_size, trainable=True)
        self.gru = tf.keras.layers.GRU(hidden_size, 
                                       return_sequences=True, 
                                       return_state=True, 
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(nwords+1)

    def call(self, x, hidden):
        x = self.embedding(x)
        # passing one time step to the GRU
        output, state = self.gru(x, initial_state=hidden)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)

        return x, state

In [0]:
decoder = Decoder(nwords_ja, emb_size, hidden_size)

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [0]:
def train_step(src, tar):
    loss = 0  
    steps = tar.shape[1]
    with tf.GradientTape() as tape:
        enc_outputs, enc_hidden = encoder(src)
        dec_hidden = enc_hidden
        # Teacher forcing - feeding the target as the next input
        for t in range(steps-1):
            dec_input = tf.expand_dims(tar[:, t], 1)   
            predictions, dec_hidden = decoder(dec_input, dec_hidden)
            loss += loss_function(tar[:, t+1], predictions)

    batch_loss = loss / int(steps-1)
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
  
    return batch_loss

In [0]:
def eval_step(src, tar):
    loss = 0    
    enc_outputs, enc_hidden = encoder(src)
    dec_hidden = enc_hidden
    # Teacher forcing - feeding the target as the next input
    for t in range(tar.shape[1]-1):
        dec_input = tf.expand_dims(tar[:, t], 1)   
        predictions, dec_hidden = decoder(dec_input, dec_hidden)
        loss += loss_function(tar[:, t+1], predictions)

    batch_loss = loss / int(tar.shape[1]-1)
  
    return batch_loss

In [23]:
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()
    print ('Epoch {} start ...... '.format(epoch + 1))
    total_loss = 0
    batch = 0
    for src, tar in train_gen():
        batch_loss = train_step(src, tar)
        total_loss += batch_loss
        batch += 1

        if batch % 20 == 0:
            print('  ---- Batch {} Loss {:.4f}'.format(batch, batch_loss))
    
    # Evaluate on dev set
    dev_loss = 0
    batch_eval = 0
    for src, tar in dev_gen():
        batch_loss = eval_step(src, tar)
        dev_loss += batch_loss
        batch_eval += 1

    print('Epoch {} finished in {} seconds. Training loss {:.4f}. Evaluation loss {:.4f}.'.format(epoch + 1, time.time() - start, total_loss / batch, dev_loss / batch_eval))

Epoch 1 start ...... 
  ---- Batch 20 Loss 1.9883
  ---- Batch 40 Loss 2.0810
  ---- Batch 60 Loss 1.9898
  ---- Batch 80 Loss 2.1106
  ---- Batch 100 Loss 2.3140
  ---- Batch 120 Loss 2.2811
  ---- Batch 140 Loss 1.9175
Epoch 1 finished in 108.7589795589447 seconds. Training loss 2.0951. Evaluation loss 1.6605.
Epoch 2 start ...... 
  ---- Batch 20 Loss 1.4755
  ---- Batch 40 Loss 1.6300
  ---- Batch 60 Loss 1.5915
  ---- Batch 80 Loss 1.6946
  ---- Batch 100 Loss 1.8669
  ---- Batch 120 Loss 1.8906
  ---- Batch 140 Loss 1.5884
Epoch 2 finished in 105.40435123443604 seconds. Training loss 1.5963. Evaluation loss 1.4679.
Epoch 3 start ...... 
  ---- Batch 20 Loss 1.2794
  ---- Batch 40 Loss 1.4625
  ---- Batch 60 Loss 1.3996
  ---- Batch 80 Loss 1.5080
  ---- Batch 100 Loss 1.6892
  ---- Batch 120 Loss 1.6955
  ---- Batch 140 Loss 1.4358
Epoch 3 finished in 105.77278327941895 seconds. Training loss 1.4206. Evaluation loss 1.4114.
Epoch 4 start ...... 
  ---- Batch 20 Loss 1.1836
  ----

In [0]:
def translate_step(src):
    enc_outputs, enc_hidden = encoder(src)
    dec_hidden = enc_hidden
    dec_input = tf.fill([src.shape[0], 1], w2i_ja['<start>'])
    results = ['<start> '] * src.shape[0]
    for t in range(30): 
        predictions, dec_hidden = decoder(dec_input, dec_hidden)
        predicted_id = tf.argmax(predictions, axis=1).numpy()

        results = [r + i2w_ja.get(i, '<end>') + ' ' for (r, i) in zip(results, predicted_id)]
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims(predicted_id, 1)
    return results

In [0]:
def translate():
    i = 0
    for src in test_gen():
        sents = translate_step(src)
        for sent in sents:
            words = sent.split()[1:]
            s = ''
            for w in words:
                if w != '<end>':
                    s = s + w + ' '
                else:
                    break
            print (f'============= Sentence {i} =============')
            print (f'translated sentence: {s}')
            tar = ' '.join(test_ja[i].split()[1:-1])
            print (f'standard target sentence: {tar}')
            print (f'BLEU score is: {bleu(s, tar)}')
            i += 1

In [0]:
def bleu_stat(hypo, ref):
    h = hypo.strip().split()
    r = ref.strip().split()
    stat = [len(h), len(r)]
    for n in range(1, 5):
        h_ngram = Counter([tuple(h[i:i+n]) for i in range(len(h)+1-n)])
        r_ngram = Counter([tuple(r[i:i+n]) for i in range(len(r)+1-n)])
        denominator = sum(h_ngram.values())
        numerator = 0
        for g in h_ngram:
            c = min([h_ngram[g], r_ngram.get(g, 0)])
            numerator += c
        stat.append(numerator)
        stat.append(denominator)
    return stat

In [0]:
def bleu(trans, tar):
    stat = bleu_stat(trans, tar)
    if 0 in stat:
        return 0
    l = sum([math.log(n/d) for n, d in zip(stat[2::2], stat[3::2])]) / 4.0
    bp = math.exp(min([0, 1-stat[1]/stat[0]]))
    return bp * math.exp(l)

In [39]:
translate()

============= Sentence 0 =============
translated sentence: あなた は 何 で も 好き な の で す か 。 
standard target sentence: 君 は １ 日 で それ が でき ま す か 。
BLEU score is: 0
============= Sentence 1 =============
translated sentence: 彼 は 私 の 申し出 を きっぱり 断 っ た 。 
standard target sentence: 皮肉 な 笑い を 浮かべ て 彼 は 私 を 見つめ た 。
BLEU score is: 0
============= Sentence 2 =============
translated sentence: あなた は 何 で も 好き な の で す か 。 
standard target sentence: 私 たち の 出発 の 時間 が 差し迫 っ て い る 。
BLEU score is: 0
============= Sentence 3 =============
translated sentence: あなた は 何 で も 好き な の で す か 。 
standard target sentence: あなた は 午後 何 を し た い で す か 。
BLEU score is: 0.2620251007173263
============= Sentence 4 =============
translated sentence: あなた の 作文 は どれ で す か 。 
standard target sentence: いつ 仕事 が 終わ る 予定 で す か 。
BLEU score is: 0.26708679784499234
============= Sentence 5 =============
translated sentence: あなた の 作文 は どれ で も 好き だ 。 
standard target sentence: 件 の 一 件 で メール を いただ き ありがとう ござ い ま し た 。
BLEU score is: 0
=====